In [69]:
import pandas as pd

import os
import sys
sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")

import requests
import time
import datetime as datetime

from bs4 import BeautifulSoup

from google.cloud import bigquery

from utils.scraping_and_api_utils import drop_dot_make_int

In [3]:
# Setting up the client
client = bigquery.Client.from_service_account_json(os.environ.get('JSON_KEY'))

# Getting the names of the projects and datasets
project = os.environ.get('PROJECT')
dataset = os.environ.get('DATASET')
links_table = os.environ.get('LINKS_TABLE')
game_data_table = os.environ.get('GAME_DATA_TABLE')
missed_data_table = os.environ.get('MISSED_DATA_TABLE')

# Creating the query and running it
query = f"""SELECT * FROM `legendary-game-recs.game_data_01_24.missed_data`"""
query_job = client.query(query)
results = query_job.result()

# Turning our Query into a DF
missed_data = results.to_dataframe()

In [6]:
frame = missed_data.drop_duplicates().copy()
frame.shape

In [8]:
# Setting count for montoring progress
# Only the first 116928 are relevant
count = 0

for game in frame['link']:
    # Monitoring progress
    if count % 10 == True:
        print(count)
    else:
        pass

    count += 1

    try:
        # Setting up the html parser + beautful soup
        game_response = requests.get(f"https://www.backloggd.com{game}")
        game_soup = BeautifulSoup(game_response.content, 'html.parser')


        # Scraping game_id
        game_id = game_soup.find('div', class_='card mx-auto game-cover overlay-hide')['game_id']


        # Scraping the title
        title = game_soup.find('h1', class_='mb-0').string

        # Scraping the date, and making it DateTime
        date_step = game_soup.find('div', class_='col-auto mt-auto pr-0')
        date_step2 = date_step.find('a', href=True).string
        if date_step2 == 'TBD':
            date_as_datetime = datetime.datetime(1, 1, 1).strftime("%Y-%m-%d")

        else:
            date_as_datetime = datetime.datetime.strptime(date_step2, '%b %d, %Y').strftime('%Y-%m-%d')


        # Getting the plays, playing, backlogs and wishlist information

        counter = game_soup.find('div', id='log-counters').find_all('a', class_='plays-counter')

        plays = drop_dot_make_int(counter[0].find('p', class_='mb-0').string.replace('K', '000'))
        playing = drop_dot_make_int(counter[1].find('p', class_='mb-0').string.replace('K', '000'))
        backlogs = drop_dot_make_int(counter[2].find('p', class_='mb-0').string.replace('K', '000'))
        wishlist = drop_dot_make_int(counter[3].find('p', class_='mb-0').string.replace('K', '000'))

        # Get a list of publis
        publisher_list = []
        try:
            publishers = game_soup.find('div', class_='col-auto pl-lg-1 sub-title').find_all('a', href=True)
            for i in range(len(publishers)):
                publisher_list.append(publishers[i].string)
        except:
            publisher_list = []

        # Get Average review score
        review_score = game_soup.find('h1', class_='text-center').string
        if review_score == 'N/A':
            review_score = 0
        else:
            review_score = float(review_score)

        # Get genres
        genres = game_soup.find_all('p', class_='genre-tag')
        genre_list = []
        for i in range(len(genres)):
            genre_list.append(genres[i].string)

        # Get platforms
        platforms = game_soup.find_all('a', class_='game-page-platform')
        platform_list = []
        for i in range(len(platforms)):
            platform_list.append(platforms[i].get_text(strip=True))

        # Get description
        description = game_soup.find('div', id='collapseSummary').get_text(strip=True)

        # Get number of reviews - number of lists associated is here as well
        lists_reviews = game_soup.find_all('p', class_='game-page-sidecard')

        total_lists = drop_dot_make_int(lists_reviews[0].get_text(strip=True).strip(" Lists").replace('K', '000'))
        total_reviews = drop_dot_make_int(lists_reviews[1].get_text(strip=True).strip(" Reviews").replace('K', '000'))

        # Get game category + main (If applicable)
        # If the search for category fails, then the game is the main game
        try:
            main_game = game_soup.find('p', class_='mb-2 game-parent-category').find('a').get_text()
            full_sentence = game_soup.find('p', class_='mb-2 game-parent-category').get_text()
            category = full_sentence.replace(main_game, '').strip()
        except:
            main_game = title
            category = 'main'

        # Get ratings, ten categories from 0.5 to 5.0

        ratings = game_soup.find_all('div', class_="col px-0 top-tooltip")

        ratings_zero_five = int(ratings[0]['data-tippy-content'].split(' |')[0])
        ratings_one_zero = int(ratings[1]['data-tippy-content'].split(' |')[0])
        ratings_one_five = int(ratings[2]['data-tippy-content'].split(' |')[0])
        ratings_two_zero = int(ratings[3]['data-tippy-content'].split(' |')[0])
        ratings_two_five = int(ratings[4]['data-tippy-content'].split(' |')[0])
        ratings_three_zero = int(ratings[5]['data-tippy-content'].split(' |')[0])
        ratings_three_five = int(ratings[6]['data-tippy-content'].split(' |')[0])
        ratings_four_zero = int(ratings[7]['data-tippy-content'].split(' |')[0])
        ratings_four_five = int(ratings[8]['data-tippy-content'].split(' |')[0])
        ratings_five_zero = int(ratings[9]['data-tippy-content'].split(' |')[0])

        # Creating a dictionary to insert into GCP
        data_dict = {'title': title,
                        'release_date': date_as_datetime,
                        'plays':plays,
                        'playing':playing,
                        'backlogs':backlogs,
                        'wishlist':wishlist,
                        'developers':str(publisher_list),
                        'avg_review':review_score,
                        'genres':str(genre_list),
                        'platforms':str(platform_list),
                        'description':description,
                        'total_reviews':total_reviews,
                        'total_lists':total_lists,
                        'category':category,
                        'main':main_game,
                        'ratings_zero_five':ratings_zero_five,
                        'ratings_one_zero':ratings_one_zero,
                        'ratings_one_five':ratings_one_five,
                        'ratings_two_zero':ratings_two_zero,
                        'ratings_two_five':ratings_two_five,
                        'ratings_three_zero':ratings_three_zero,
                        'ratings_three_five':ratings_three_five,
                        'ratings_four_zero':ratings_four_zero,
                        'ratings_four_five':ratings_four_five,
                        'ratings_five_zero':ratings_five_zero,
                        'url':game,
                        'game_id':game_id}

        # Inserting into GCP
        table = client.dataset(dataset).table('game_data_test')
        errors = client.insert_rows_json(table, [data_dict])

        # Relaying errors for some sexy debugging
        if errors:
            print(f'Failed {game}, {errors}')
            #Saving missed data, so we can try get it later
            table = client.dataset(dataset).table('missed_data_2nd')
            errors = client.insert_rows_json(table, [{'link':game}])
        else:
            print(f'all good {count}')


    except:
        # Relaying when Backlogged rejects us
        print(f'Page failure at {game}, count = {count}')
        #Saving missed data, so we can try get it later
        table = client.dataset(dataset).table('missed_data_2nd')
        errors = client.insert_rows_json(table, [{'link':game}])

        time.sleep(3)


all good 1
1
all good 2
all good 3
all good 4
all good 5
all good 6
all good 7
all good 8
all good 9
all good 10
all good 11
11
all good 12
all good 13
all good 14
all good 15
all good 16
all good 17
all good 18
all good 19
all good 20
all good 21
21
all good 22
all good 23
all good 24
all good 25
all good 26
all good 27
all good 28
all good 29
all good 30
all good 31
31
all good 32
all good 33
all good 34
all good 35
all good 36
all good 37
all good 38
all good 39
all good 40
all good 41
41
all good 42
all good 43
all good 44
all good 45
all good 46
all good 47
all good 48
all good 49
all good 50
all good 51
51
all good 52
all good 53
all good 54
all good 55
all good 56
all good 57
all good 58
all good 59
all good 60
all good 61
61
all good 62
all good 63
all good 64
all good 65
all good 66
all good 67
all good 68
all good 69
all good 70
all good 71
71
all good 72
all good 73
all good 74
all good 75
all good 76
all good 77
all good 78
all good 79
all good 80
all good 81
81
all good 82

# API Section

In [1]:
import os
import sys
import pandas as pd
import requests
from igdb.wrapper import IGDBWrapper
import json
sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")

from google.cloud import bigquery

from utils.processing_utils import process_raw_data
from utils.scraping_and_api_utils import prepare_json_df

from preprocessing.api_processing import *

import time

In [2]:
# List of games used in the model
client = bigquery.Client.from_service_account_json(os.environ.get('JSON_KEY'))

project = os.environ.get('PROJECT')
dataset = os.environ.get('DATASET')
links_table = os.environ.get('LINKS_TABLE')
game_data_table = os.environ.get('GAME_DATA_TABLE')
missed_data_table = os.environ.get('MISSED_DATA_TABLE')
test_table = 'test_table'

query_1 = f"""SELECT * FROM `legendary-game-recs.game_data_01_24.game_data`"""
query_2 = f"""SELECT * FROM `legendary-game-recs.game_data_01_24.game_data_test`"""

# query = f"""SELECT * FROM `{project}.{dataset}.{links_table}"""
query_job_1 = client.query(query_1)
results_1 = query_job_1.result()
game_data_1 = results_1.to_dataframe()

query_job_2 = client.query(query_2)
results_2 = query_job_2.result()
game_data_2 = results_2.to_dataframe()

data = pd.concat([game_data_1, game_data_2]).reset_index(drop=True)

re, up = process_raw_data(data, 2024, 1, 24)

all_data = pd.concat([re,up])

list_of_game_id = all_data['game_id']

In [3]:
all_data.shape, data.shape

((36991, 26), (118656, 27))

In [34]:
table_ref = client.dataset('game_data_01_24').table('all_data')
tabless = client.get_table(table_ref)

for field in tabless.schema:
    print(f"{field.name}: {field.field_type},")

title: STRING,
release_date: TIMESTAMP,
plays: INTEGER,
playing: INTEGER,
backlogs: INTEGER,
wishlist: INTEGER,
developers: STRING,
avg_review: FLOAT,
genres: STRING,
platforms: STRING,
description: STRING,
total_reviews: INTEGER,
total_lists: INTEGER,
main: STRING,
ratings_zero_five: INTEGER,
ratings_one_zero: INTEGER,
ratings_one_five: INTEGER,
ratings_two_zero: INTEGER,
ratings_two_five: INTEGER,
ratings_three_zero: INTEGER,
ratings_three_five: INTEGER,
ratings_four_zero: INTEGER,
ratings_four_five: INTEGER,
ratings_five_zero: INTEGER,
url: STRING,
game_id: INTEGER,
id: FLOAT,
themes: STRING,
franchises: STRING,
game_modes: STRING,
player_perspectives: STRING,
franchise: STRING,
storyline: STRING,
multiplayer_modes: STRING,
aggregated_rating: FLOAT,
aggregated_rating_count: FLOAT,
rating: FLOAT,
game_engines: STRING,


In [5]:
# Credentials for calling the API
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
GRANT_TYPE = os.getenv("GRANT_TYPE")

#Generating the access token
response = requests.post(f'https://id.twitch.tv/oauth2/token?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&grant_type={GRANT_TYPE}')
response_json = response.json()
ACCESS_TOKEN = response_json['access_token']


#This is a wrapper from IGDB just for their API
wrapper = IGDBWrapper(CLIENT_ID, ACCESS_TOKEN)

no_data = []
list_sons = []
count = 0

for game in list_of_game_id[:500]:
    count += 1

    try:
        the_feat = json.loads(wrapper.api_request('games',
                f'fields franchise, franchises, storyline, aggregated_rating,aggregated_rating_count, game_engines.name, game_modes.name, multiplayer_modes, player_perspectives.name, themes.name, rating; where id = {int(game)};'))
        list_sons.append(the_feat[0])
        print(f'Success at {count}')

    except:
        print(f'failed at {count}')
        no_data.append(game)

    time.sleep(0.125)

Success at 1
Success at 2
Success at 3
Success at 4
Success at 5
Success at 6
Success at 7
failed at 8
Success at 9
Success at 10
Success at 11
Success at 12
Success at 13
Success at 14
Success at 15
Success at 16
Success at 17
Success at 18
Success at 19
Success at 20
Success at 21
Success at 22
Success at 23
Success at 24
Success at 25
Success at 26
Success at 27
Success at 28
Success at 29
Success at 30
Success at 31
Success at 32
Success at 33
Success at 34
failed at 35
Success at 36
Success at 37
Success at 38
Success at 39
Success at 40
Success at 41
Success at 42
Success at 43
Success at 44
Success at 45
Success at 46
Success at 47
Success at 48
Success at 49
Success at 50
Success at 51
Success at 52
Success at 53
failed at 54
Success at 55
Success at 56
Success at 57
Success at 58
Success at 59
Success at 60
Success at 61
Success at 62
Success at 63
Success at 64
Success at 65
Success at 66
Success at 67
Success at 68
Success at 69
Success at 70
Success at 71
Success at 72
Succ

In [6]:
missed = pd.DataFrame(no_data, columns=['game_id'])

In [7]:
# This is used for getting the information from age_ratings.
def get_category_descriptions(json_dict):
    category_list = []

    if type(json_dict) == float:
        category_list = None
    else:
        for i in range(len(json_dict)):
            try:
                descriptions = json_dict[i]['content_descriptions']
                for i in range(len(descriptions)):
                    rating_descript = descriptions[i]['category']
                    category_list.append(rating_descript)

            except:
                pass

    return category_list

# This goes through each key in the JSON and creates a list of items for the variable
def get_names_from_dict(dict_list):
    list_of_items = []
    try:
        for i in range(len(dict_list)):
            item = dict_list[i]['name']
            list_of_items.append(item)
    except:
        list_of_items = None

    return list_of_items

# This uses the above functions to create a final, clean df
def prepare_json_df(original_json_df):
    original_json_df['themes'] = original_json_df['themes'].apply(get_names_from_dict)
    original_json_df['game_modes'] = original_json_df['game_modes'].apply(get_names_from_dict)
    original_json_df['player_perspectives'] = original_json_df['player_perspectives'].apply(get_names_from_dict)
    original_json_df['game_engines'] = original_json_df['game_engines'].apply(get_names_from_dict)
    # original_json_df['age_ratings'] = original_json_df['age_ratings'].apply(get_category_descriptions)

    return original_json_df

In [8]:
# Creates a df from the list of dictionaries
new_df = prepare_json_df(pd.DataFrame(list_sons))

In [31]:
everything = all_data.merge(new_df, right_on='id', left_on='game_id', how='left')

In [32]:
# data[data['game_id'].isin(missed['game_id'])].sort_values(by='plays')

In [33]:
# You need to ensure the categorical columns can go in as strings
# Because BiqQuery treats it like a JSON
list_of_lists = ['game_modes', 'franchises', 'franchise', 'player_perspectives', 'themes', 'multiplayer_modes', 'game_engines']
everything[list_of_lists] = everything[list_of_lists].astype(str)

#Uploads to the cloud
job_config = bigquery.LoadJobConfig(write_disposition='WRITE_TRUNCATE')
job = client.load_table_from_dataframe(everything, 'legendary-game-recs.game_data_01_24.all_data', job_config=job_config)

#Uploads to the cloud
job_config = bigquery.LoadJobConfig(write_disposition='WRITE_TRUNCATE')
job = client.load_table_from_dataframe(missed, 'legendary-game-recs.game_data_01_24.missed_api_data', job_config=job_config)
